In [ ]:
# There are 3 types of recommendation system
# 1 Content based recommendation
# 2 Popularity based recommendation
# 3 Collaborative recommendation system 

In [38]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
warnings.warn('Alert')
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
#Data collection and analysis
df = pd.read_csv('movies.csv')

In [70]:
df.head(69)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,64,178000000,Science Fiction,http://www.foxmovies.com/movies/x-men-apocalypse,246655,mutant supernatural powers marvel comic superh...,en,X-Men: Apocalypse,After the re-emergence of the world's first mu...,139.272042,...,144.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Only the strong will survive,X-Men: Apocalypse,6.4,4721,James McAvoy Michael Fassbender Jennifer Lawre...,"[{'name': 'Grant Major', 'gender': 2, 'departm...",Bryan Singer
65,65,185000000,Drama Action Crime Thriller,http://thedarkknight.warnerbros.com/dvdsite/,155,dc comics crime fighter secret identity scarec...,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,187.322927,...,152.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Why So Serious?,The Dark Knight,8.2,12002,Christian Bale Heath Ledger Aaron Eckhart Mich...,"[{'name': 'Lucinda Syson', 'gender': 1, 'depar...",Christopher Nolan
66,66,175000000,Animation Comedy Family Adventure,http://disney.go.com/disneypictures/up/,14160,age difference central and south america ballo...,en,Up,Carl Fredricksen spent his entire life dreamin...,92.201962,...,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,,Up,7.7,6870,Ed Asner Christopher Plummer Jordan Nagai Bob ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Pete Docter
67,67,175000000,Animation Family Adventure Science F

In [42]:
#no of rows and columns
df.shape

(4803, 24)

In [43]:
df['original_language'].value_counts()

original_language
en    4505
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
cn      12
ru      11
ko      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: count, dtype: int64

In [44]:
#selecting the relevant features for recommendation
selected_features = ['genres','keywords','tagline','original_language', 'popularity', 'vote_average', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'original_language', 'popularity', 'vote_average', 'cast', 'director']


In [45]:
# replacing null values with null strings
for feature in selected_features:
    df[feature] = df[feature].fillna('')

In [46]:
#All the missing values have being replace
df[feature].isnull().sum()

0

In [47]:
# combining all the selected features
#concartinating only txt not str
combined_features = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['original_language']+' '+df['cast']+' '+df['director']

In [48]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai en Daniel Henney El...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [49]:
#converting the txt to data to feature data
vectorizer = TfidfVectorizer()

In [50]:
#Using vectorizer func to transform the data
feature_vectors = vectorizer.fit_transform(combined_features)

In [51]:
print(feature_vectors)

  (0, 2434)	0.17262107186206177
  (0, 7763)	0.11273627752842749
  (0, 13038)	0.19412033270357637
  (0, 10237)	0.16049105450885073
  (0, 8764)	0.22695468595009713
  (0, 14624)	0.15141634132650292
  (0, 16688)	0.19831426292450674
  (0, 14079)	0.20583803637232165
  (0, 13334)	0.2176171550320001
  (0, 17312)	0.20185863312549773
  (0, 17027)	0.2362922168681942
  (0, 13364)	0.15012303027709262
  (0, 4911)	0.03453638093174035
  (0, 11513)	0.27195076912003546
  (0, 11202)	0.09043921375601767
  (0, 17018)	0.12813614591803602
  (0, 15279)	0.07091600479711815
  (0, 4950)	0.24011519661696928
  (0, 14287)	0.21379417528322503
  (0, 3228)	0.2494527275350663
  (0, 16607)	0.12541945881301636
  (0, 14394)	0.33942491434120725
  (0, 5843)	0.16457685199337505
  (0, 3067)	0.22195129200668776
  (0, 3682)	0.21379417528322503
  :	:
  (4801, 404)	0.177156898310652
  (4801, 6942)	0.2884161588693086
  (4801, 11673)	0.21543035497777507
  (4801, 1674)	0.15637434380345686
  (4801, 10938)	0.13495105580020303
  (4801,

In [53]:
#Cosine Similarity
#Getting the similarity scores
#The cosine similarity function checks the selected features to know which are similar

similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.0731602  0.03890829 ... 0.0012222  0.00126498 0.0011916 ]
 [0.0731602  1.         0.03384574 ... 0.0367837  0.00110423 0.00104018]
 [0.03890829 0.03384574 1.         ... 0.00125046 0.05512248 0.00121915]
 ...
 [0.0012222  0.0367837  0.00125046 ... 1.         0.00129619 0.02770363]
 [0.00126498 0.00110423 0.05512248 ... 0.00129619 1.         0.00126374]
 [0.0011916  0.00104018 0.00121915 ... 0.02770363 0.00126374 1.        ]]


In [54]:
similarity.shape


(4803, 4803)

In [55]:
#Getting movie name from the user
movie_name= input('Enter your favorite movie name: ')

Enter your favorite movie name: iron man


In [61]:
# Creating list with all the movie names given in the dataset

list_of_all_titles = df['title'].tolist() 
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [63]:
#finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [66]:
close_match = find_close_match[0]
close_match

'Iron Man'

In [69]:
# Finding the index of the movie with the title
index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)

68


In [77]:
# getting list of similar movies
similarity_score=list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

#The first number is the index of the movie and the next is the similarity score

[(0, 0.03465171109216982), (1, 0.05556778067593926), (2, 0.01486454916655431), (3, 0.007709358491484988), (4, 0.03374306020417837), (5, 0.015049785150951484), (6, 0.07797319974531627), (7, 0.2403022214847842), (8, 0.009028543905665234), (9, 0.0770112494466352), (10, 0.07652233756895334), (11, 0.012908022428614139), (12, 0.014834102597908428), (13, 0.013394595300126936), (14, 0.09757508316635838), (15, 0.008346520594088644), (16, 0.22787155644031118), (17, 0.014191207564534139), (18, 0.042429788913276256), (19, 0.07972629431802007), (20, 0.08083662412805429), (21, 0.01219516759237631), (22, 0.007895988583183917), (23, 0.007560093008881947), (24, 0.013707194994274316), (25, 0.0011262310442939829), (26, 0.2165749647848831), (27, 0.03156918897521504), (28, 0.06212500384638118), (29, 0.01519993078941688), (30, 0.08180953017829944), (31, 0.3154235385094266), (32, 0.0297872938558197), (33, 0.13187295553545791), (34, 0.0011345042914415915), (35, 0.03648620525506104), (36, 0.03288457323691354),

In [78]:
len(similarity_score)

4803

In [81]:
# sorting the movies based on their similaries score in descending order

sorted_similar_movies=sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(68, 1.0000000000000002), (79, 0.40954151270129435), (31, 0.3154235385094266), (7, 0.2403022214847842), (16, 0.22787155644031118), (26, 0.2165749647848831), (85, 0.20713527449490834), (182, 0.19660400647660514), (511, 0.1679784684949355), (3623, 0.16197544802792216), (64, 0.1539618431511734), (203, 0.14904802860607627), (4401, 0.1484953518035033), (174, 0.14817618632850948), (101, 0.14504623634397906), (46, 0.14308508117134586), (169, 0.13904505487314897), (1740, 0.13719470835214767), (94, 0.13710749351386797), (788, 0.13395646909900114), (126, 0.13353494814479042), (131, 0.1324222037628412), (33, 0.13187295553545791), (2487, 0.12408039157338295), (783, 0.1225813161713326), (138, 0.11940498015581812), (2442, 0.11827134555898897), (661, 0.11822812033513744), (607, 0.11484833188235835), (38, 0.11363384282434985), (2651, 0.11309583799734033), (353, 0.11249167668176276), (122, 0.10958039249972376), (1553, 0.10889149613736333), (1451, 0.10878387936017392), (242, 0.1073626616493186), (618, 

In [82]:
len(sorted_similar_movies)

4803

In [91]:
# print the name of similar movies based on the index

print('Movie suggested for you : \n')

#creating a for loop

i=1
for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index = df[df.index==index]['title'].values[0]
    if (i<10):
        print(i, ',', title_from_index)
        i+=1

Movie suggested for you : 

1 , Iron Man
2 , Iron Man 2
3 , Iron Man 3
4 , Avengers: Age of Ultron
5 , The Avengers
6 , Captain America: Civil War
7 , Captain America: The Winter Soldier
8 , Ant-Man
9 , X-Men


In [94]:
# Movie recommndation system 
movie_name= input('Enter your favorite movie name: ')

list_of_all_titles = df['title'].tolist() 

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match = find_close_match[0]

index_of_the_movie = df[df.title == close_match]['index'].values[0]
similarity_score=list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies=sorted(similarity_score, key = lambda x:x[1], reverse = True)
print('Movie suggested for you : \n')



i=1
for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index = df[df.index==index]['title'].values[0]
    if (i<20):
        print(i, ',', title_from_index)
        i+=1

Enter your favorite movie name: Iron man
Movie suggested for you : 

1 , Iron Man
2 , Iron Man 2
3 , Iron Man 3
4 , Avengers: Age of Ultron
5 , The Avengers
6 , Captain America: Civil War
7 , Captain America: The Winter Soldier
8 , Ant-Man
9 , X-Men
10 , Made
11 , X-Men: Apocalypse
12 , X2
13 , The Helix... Loaded
14 , The Incredible Hulk
15 , X-Men: First Class
16 , X-Men: Days of Future Past
17 , Captain America: The First Avenger
18 , Kick-Ass 2
19 , Guardians of the Galaxy
